# What is the average revenue of the orders?

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("exercise1").getOrCreate()

2021-09-25 14:51:26,832 WARN util.Utils: Your hostname, tb-LinuxBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
2021-09-25 14:51:26,834 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
2021-09-25 14:51:29,329 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
orders_data = "file:///home/tamaghna/big_data_spark/sales_parquet"

In [3]:
from pyspark.sql.types import StructType, IntegerType, StringType, DateType
from pyspark.sql.functions import to_date

In [8]:
# orders_schema = StructType().add()
#                             .add("order_id",IntegerType(),True)\
#                             .add("product_id",IntegerType(),True)\
#                             .add("seller_id",IntegerType(),True)\
#                             .add("date",DateType(),True)\
#                             .add("num_pieces_sold",IntegerType(),True)\
#                             .add("bill_raw_text",StringType(),True)

In [45]:
orders = spark.read.format("parquet")\
        .options(header=True)\
        .load(orders_data)

In [48]:
orders.count()

20000040

In [47]:
orders = orders.drop("bill_raw_text")

In [50]:
orders.show()

+--------+----------+---------+----------+---------------+
|order_id|product_id|seller_id|      date|num_pieces_sold|
+--------+----------+---------+----------+---------------+
|       1|         0|        0|2020-07-10|             26|
|       2|         0|        0|2020-07-08|             13|
|       3|         0|        0|2020-07-05|             38|
|       4|         0|        0|2020-07-05|             56|
|       5|         0|        0|2020-07-05|             11|
|       6|         0|        0|2020-07-01|             82|
|       7|         0|        0|2020-07-04|             15|
|       8|         0|        0|2020-07-08|             79|
|       9|         0|        0|2020-07-10|             25|
|      10|         0|        0|2020-07-08|              8|
|      11|         0|        0|2020-07-01|             10|
|      12|         0|        0|2020-07-06|             45|
|      13|         0|        0|2020-07-10|             63|
|      14|         0|        0|2020-07-03|             2

In [51]:
orders = orders.withColumn("order_id",orders.order_id.cast(IntegerType()))\
        .withColumn("product_id",orders.product_id.cast(IntegerType()))\
        .withColumn("seller_id",orders.seller_id.cast(IntegerType()))\
        .withColumn("date",to_date(orders.date,"yyyy-MM-dd"))\
        .withColumn("num_pieces_sold",orders.num_pieces_sold.cast(IntegerType()))

In [52]:
products = spark.read.format("parquet")\
                        .options(header=True)\
                        .load("file:///home/tamaghna/big_data_spark/products_parquet")

In [53]:
products.count()

75000000

In [54]:
products.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- price: string (nullable = true)



In [55]:
from pyspark.sql.types import IntegerType,StringType,FloatType

In [56]:
products.filter(products.product_id.isNull()).show()

+----------+------------+-----+
|product_id|product_name|price|
+----------+------------+-----+
+----------+------------+-----+



In [57]:
products = products.withColumn("product_id",products.product_id.cast(IntegerType()))\
                    .withColumn("product_name",products.product_name.cast(StringType()))\
                    .withColumn("price",products.price.cast(FloatType()))

In [58]:
products.show()

+----------+------------+-----+
|product_id|product_name|price|
+----------+------------+-----+
|         0|   product_0| 22.0|
|         1|   product_1| 30.0|
|         2|   product_2| 91.0|
|         3|   product_3| 37.0|
|         4|   product_4|145.0|
|         5|   product_5|128.0|
|         6|   product_6| 66.0|
|         7|   product_7|145.0|
|         8|   product_8| 51.0|
|         9|   product_9| 44.0|
|        10|  product_10| 53.0|
|        11|  product_11| 13.0|
|        12|  product_12|104.0|
|        13|  product_13|102.0|
|        14|  product_14| 24.0|
|        15|  product_15| 14.0|
|        16|  product_16| 38.0|
|        17|  product_17| 72.0|
|        18|  product_18| 16.0|
|        19|  product_19| 46.0|
+----------+------------+-----+
only showing top 20 rows



In [59]:
ord_prod_join = orders.join(products,on='product_id')

In [60]:
ord_prod_join.show()

+----------+--------+---------+----------+---------------+--------------+-----+
|product_id|order_id|seller_id|      date|num_pieces_sold|  product_name|price|
+----------+--------+---------+----------+---------------+--------------+-----+
|     33602|12992813|        3|2020-07-07|             61| product_33602|143.0|
|     35947| 6489803|        9|2020-07-10|             88| product_35947| 40.0|
|     53634|14993817|        3|2020-07-04|              2| product_53634| 87.0|
|     64590| 4978574|        5|2020-07-01|             84| product_64590|113.0|
|     74757|18490481|        4|2020-07-05|             52| product_74757|115.0|
|     99621| 6488801|        3|2020-07-02|             12| product_99621| 45.0|
|    102594|12494938|        5|2020-07-04|             66|product_102594|131.0|
|    141533|12975463|        8|2020-07-01|             42|product_141533| 69.0|
|    197949| 8986189|        4|2020-07-09|             45|product_197949|148.0|
|    200625| 1992704|        7|2020-07-0

In [61]:
order_revenue = ord_prod_join.withColumn("revenue",ord_prod_join.num_pieces_sold*ord_prod_join.price)

In [62]:
order_revenue.show()

+----------+--------+---------+----------+---------------+--------------+-----+-------+
|product_id|order_id|seller_id|      date|num_pieces_sold|  product_name|price|revenue|
+----------+--------+---------+----------+---------------+--------------+-----+-------+
|     33602|12992813|        3|2020-07-07|             61| product_33602|143.0| 8723.0|
|     35947| 6489803|        9|2020-07-10|             88| product_35947| 40.0| 3520.0|
|     53634|14993817|        3|2020-07-04|              2| product_53634| 87.0|  174.0|
|     64590| 4978574|        5|2020-07-01|             84| product_64590|113.0| 9492.0|
|     74757|18490481|        4|2020-07-05|             52| product_74757|115.0| 5980.0|
|     99621| 6488801|        3|2020-07-02|             12| product_99621| 45.0|  540.0|
|    102594|12494938|        5|2020-07-04|             66|product_102594|131.0| 8646.0|
|    141533|12975463|        8|2020-07-01|             42|product_141533| 69.0| 2898.0|
|    197949| 8986189|        4|2

In [64]:
order_revenue.agg(avg("revenue")).show()

+------------------+
|      avg(revenue)|
+------------------+
|1246.1338560822878|
+------------------+



In [40]:
order_revenue.groupBy("order_id").mean("revenue").show()

+--------+------------+
|order_id|avg(revenue)|
+--------+------------+
|       1|       572.0|
|       2|       286.0|
|       3|       836.0|
|       4|      1232.0|
|       5|       242.0|
|       6|      1804.0|
|       7|       330.0|
|       8|      1738.0|
|       9|       550.0|
|      10|       176.0|
|      11|       220.0|
|      12|       990.0|
|      13|      1386.0|
|      14|       484.0|
|      15|      1650.0|
|      16|      1826.0|
|      17|      1188.0|
|      18|      1276.0|
|      19|       726.0|
|      20|      1606.0|
+--------+------------+
only showing top 20 rows



In [42]:
from pyspark.sql.functions import *

In [43]:
orders.join(products, orders["product_id"] == products["product_id"], "inner").\
      agg(avg(products["price"] * orders["num_pieces_sold"])).show()

+------------------------------+
|avg((price * num_pieces_sold))|
+------------------------------+
|                      1104.202|
+------------------------------+



In [ ]:
products.select("product_id").collect()